In [2]:
from PyPDF2 import PdfReader
from transformers import pipeline
from docx import Document

# 1. Trích xuất văn bản từ file PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# 2. Tóm tắt văn bản bằng mô hình Transformers
def summarize_text(text, model_name="facebook/bart-large-cnn"):
    summarizer = pipeline("summarization", model=model_name)
    max_input_length = 1024  # BERTSUM/PEGASUS thường giới hạn độ dài input
    summaries = []
    
    # Chia văn bản dài thành đoạn nhỏ để tóm tắt
    for i in range(0, len(text), max_input_length):
        chunk = text[i:i + max_input_length]
        summary = summarizer(chunk, max_length=130, min_length=30, do_sample=False)
        summaries.append(summary[0]['summary_text'])
    
    return "\n".join(summaries)

# 3. Lưu nội dung tóm tắt vào file DOCX
def save_to_docx(summary, output_path):
    doc = Document()
    doc.add_heading("Summary of the PDF", level=1)
    doc.add_paragraph(summary)
    doc.save(output_path)

# --- Main Function ---
if __name__ == "__main__":
    pdf_path = "PDF/0e21835a42a6df2405496f62647058ff855743c1.pdf"  # Đường dẫn tới file PDF của bạn
    output_docx = "output_summary.docx"  # Tên file DOCX xuất ra
    
    print("Đang trích xuất văn bản từ PDF...")
    pdf_text = extract_text_from_pdf(pdf_path)
    
    print("Đang tóm tắt văn bản...")
    summarized_text = summarize_text(pdf_text)
    
    print("Đang lưu tóm tắt vào file DOCX...")
    save_to_docx(summarized_text, output_docx)
    
    print(f"Tóm tắt hoàn thành! File DOCX đã lưu tại: {output_docx}")


Đang trích xuất văn bản từ PDF...
Đang tóm tắt văn bản...


Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Đang lưu tóm tắt vào file DOCX...
Tóm tắt hoàn thành! File DOCX đã lưu tại: output_summary.docx


In [ ]:
import google.generativeai as genai
import httpx
import base64

model = genai.GenerativeModel("gemini-1.5-flash")
doc_url = "https://discovery.ucl.ac.uk/id/eprint/10089234/1/343019_3_art_0_py4t4l_convrt.pdf"  # Replace with the actual URL of your PDF

# Retrieve and encode the PDF
doc_data = base64.standard_b64encode(httpx.get(doc_url).content).decode("utf-8")

prompt = "Summarize this document"

response = model.generate_content([{'mime_type':'application/pdf', 'data': doc_data}, prompt])
print(response.text)